<a href="https://colab.research.google.com/github/meiaalsup/lda_infographics/blob/master/6_804.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

6.804 Final Project

Meia Alsup, Sravya Bhamidipati, Anelise Newman

In [1]:
from string import digits
import json
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image, display
from IPython.core.display import HTML 
import urllib.request as request

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import nltk

In [2]:
#f = open('raw_ocr_output.pickle','rb')
#ocr = pkl.load(f)
#f.close()

In [3]:
#less_ocr = {key: ocr[key] for i, key in enumerate(ocr) if i < 100}
#less_ocr

In [4]:
#f = open('icon_annotations_testing.pickle','rb')
#icons = pkl.load(f)
#f.close()

In [5]:
# annotations from task where participants marked bounding boxes around all icons on an infographic
icon_dict = pkl.load(open( "icon_annotations_all.pickle", "rb") )

In [6]:
len(icon_dict)

1400

In [7]:
icons_english  = set(icon_dict.keys())
#icons_english

In [8]:
NUM_TOPICS = 30

In [10]:
f = open('google_text_extraction_output.pckl','rb')
infographic_name_to_words = pkl.load(f)
f.close()

print('We have extracted words from %d infographics.'%len(infographic_name_to_words))

We have extracted words from 62361 infographics.


In [11]:
def contains_english(set_of_words):
    english = set(['the', 'she', 'different', 'time', 'and', 'client', 'sports', 'grammar', 'family', 'report'])
    spanish = set(['el', 'es', 'cuando', 'esta', 'malo', 'que', 'tiene', 'para'])
    if bool(set(set_of_words) & spanish):
        return False
    if bool(set(set_of_words) & english):
        return True
    return False

In [12]:
english  = {key: val for key, val in infographic_name_to_words.items() if contains_english(val)}
len(english)

46952

In [14]:
#english.keys()

In [11]:
#json.dump(english, open('filtered.json', 'w'))
#json.dump([key for key, val in english.items()], open('filtered_keys.json', 'w'))

In [19]:
words = [list_words for list_words in english.values()]
newList = []
stemmer = PorterStemmer()
for wordList in words:
    thisList = []
    for word in wordList:
        if len(word) > 2 and word.isalpha() and word not in STOPWORDS and wordnet.synsets(word):
            thisList.append(word.lower())
            #thisList.append(stemmer.stem(WordNetLemmatizer().lemmatize(word.lower(), pos='v')))
    newList.append(thisList)

In [20]:
len(newList)

46952

In [21]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(newList)
common_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

common_corpus = [common_dictionary.doc2bow(text) for text in newList]
# Train the model on the corpus.
id2token = {val:key for key, val in common_dictionary.token2id.items()}

In [22]:
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=NUM_TOPICS)

In [23]:
for i in range(NUM_TOPICS):
    terms = lda.get_topic_terms(i, 20)
    print(f'Topic number {i}')
    val = {id2token[t[0]]: t[1] for t in terms}
    #print(f'Top 20 terms: {val}')
    print(f'Top 20 terms: {val.keys()}')

Topic number 0
Top 20 terms: dict_keys(['skin', 'yes', 'cancer', 'are', 'breast', 'crime', 'cases', 'people', 'deaths', 'death', 'can', 'year', 'smoking', 'tobacco', 'radiation', 'smoke', 'not', 'criminal', 'common', 'years'])
Topic number 1
Top 20 terms: dict_keys(['job', 'work', 'employees', 'jobs', 'security', 'employee', 'workers', 'training', 'company', 'management', 'working', 'office', 'professional', 'companies', 'career', 'skills', 'salary', 'services', 'manager', 'employers'])
Topic number 2
Top 20 terms: dict_keys(['water', 'home', 'real', 'property', 'estate', 'solar', 'homes', 'house', 'average', 'price', 'gallons', 'rent', 'bed', 'housing', 'cost', 'properties', 'year', 'bugs', 'prices', 'buyers'])
Topic number 3
Top 20 terms: dict_keys(['social', 'media', 'business', 'twitter', 'marketing', 'businesses', 'small', 'use', 'brand', 'customers', 'local', 'online', 'post', 'users', 'companies', 'followers', 'posts', 'service', 'people', 'customer'])
Topic number 4
Top 20 term

In [24]:
saved = lda.save('saved_lda_30_topics')

In [36]:
list(english.keys())[:10]

['free-autism-infographic_564c4ed54ea8f.jpg',
 'lemonlys-one-sweet-year-2012-annual-report_51141ef25ba83.jpg',
 'top-10-darkest-characters-in-film_544fde05817b7.jpg',
 '12-christmas-traditions-from-around-the-world_50290c852911e.jpg',
 'dc-power-on-board-lowering-environmental-impact_5029160e8b05c.png',
 'common-health-issues-veterans-face_5385e1a9af293.jpg',
 '6-classic-coat-styles-for-men_5306499534889.jpg',
 'interesting-facts-about-teeth_53d291913ac5f.jpg',
 'databagg--online-storage-service_53cf95209c581.jpg',
 'diabetes-month-an-infographic_52791b9fc14d8.jpg']

In [17]:
base='http://thumbnails.visually.netdna-cdn.com'

In [15]:

for url_ in list(english.keys())[30:150]:
    #display(Image(url=f'{base}/{url_}'))
    request.urlretrieve(f'{base}/{url_}', f'images/{url_}')


NameError: name 'english' is not defined

In [13]:
intersection = set([url_ for url_ in english.keys() if url_ in icons_english])

In [15]:
len(intersection)

1079

In [18]:
for url_ in intersection:
    #display(Image(url=f'{base}/{url_}'))
    request.urlretrieve(f'{base}/{url_}', f'images/{url_}')